In [1]:
import pandas as pd
import numpy as np
import pickle
import math

In [2]:
# This first part of the procedure works for all TUT datasets, that have heterogeneous characteristics

In [3]:
#                                                                                                             #                              
#                                                                                                             #   
####################################### PART FOR TUT3 AND TUT4 DATASETS ####################################### 
#                                                                                                             #   
#                                                                                                             #   

In [9]:
dataset_name = 'TUT4' #TUT3 #TUT4


work_path = 'raw_db/' + dataset_name + '/'








# Generating devices file
def format_manufacturer(oldname):
    if oldname == 'HTC':
        return oldname
    if oldname == 'asus':
        return oldname.upper()
    if oldname == 'Letv':
        return 'LeTV'
    if oldname == 'LGE':
        return oldname
    else:
        return oldname.title()
    


tmp = pd.read_csv(work_path + '/Test_device_21Aug17.csv', sep=',', header=None)
tmp = pd.concat([tmp, pd.read_csv(work_path + '/Training_device_21Aug17.csv', sep=',', header=None)], axis=0)
tmp = tmp.drop_duplicates().reset_index()
tmp.columns = ['device_id', 'old_name']
tmp['device_model'] = [' '.join(x.strip().split(' ')[1:]) if 'LENOVO' not in x else ' '.join(x.strip().split(' ')[2:]) for x in tmp['old_name']]
tmp['device_manufacturer'] = [format_manufacturer(x.strip().split(' ')[0]) for x in tmp['old_name']]
tmp['device_type'] = 'smartphone'
tmp['notes'] = 'NULL'

tmp[['device_id', 'device_model', 'device_manufacturer', 'device_type', 'notes']].to_csv(work_path + '/devices.csv', header=True, index=None)

oldname_map_id = {}
for idx, old in tmp.iterrows():
    oldname_map_id[old.old_name] = old.device_id

display(tmp)



train_split_name = ''
test_split_name = ''
if dataset_name == 'TUT3':
    train_split_name = 'Training'
    test_split_name = 'Test'
elif dataset_name == 'TUT4':
    train_split_name = 'Test'
    test_split_name = 'Training'





# Training data 
dataframe = pd.read_csv(work_path + '/' + train_split_name + '_coordinates_21Aug17.csv', sep=',', header=None)
dataframe.columns = ['coord_x', 'coord_y', 'coord_z']
floor_heights = [0.0, 3.7, 7.4, 11.1, 14.8]
dataframe['floor'] = [floor_heights.index(x) for x in dataframe['coord_z']]
dataframe['coord_z'] = 'NULL'
dataframe['building'] = '0'
dataframe['site'] = 'NULL'
dataframe['tile'] = 'crowd_tile_1'
dataframe['user_id'] = 'NULL'
dataframe['device_id'] = [oldname_map_id[x] for x in pd.read_csv(work_path +'/' + train_split_name + '_device_21Aug17.csv', sep=',', header=None).iloc[:,0]] 

tmp = pd.read_csv(work_path + '/' + train_split_name + '_date_21Aug17.csv', sep=',', header=None)
tmp.columns = ['epoch']
dataframe = pd.concat([dataframe, tmp], axis=1)

tmp = pd.read_csv(work_path + '/' + train_split_name + '_rss_21Aug17.csv', sep=',', header=None)
tmp[[x for x in tmp.columns]] = tmp[[x for x in tmp.columns]].replace(100, 'NULL')
tmp.columns = ['WAP000'[:-len(str(x))] + str(x) for x in tmp.columns]

dataframe = pd.concat([tmp, dataframe], axis=1)

dataframe['set'] = 'training'




# Test data (which is the training data of TUT4)
dataframe_test = pd.read_csv(work_path + '/' + test_split_name + '_coordinates_21Aug17.csv', sep=',', header=None)
dataframe_test.columns = ['coord_x', 'coord_y', 'coord_z']
floor_heights = [0.0, 3.7, 7.4, 11.1, 14.8]
dataframe_test['floor'] = [floor_heights.index(x) for x in dataframe_test['coord_z']]
dataframe_test['coord_z'] = 'NULL'
dataframe_test['building'] = '0'
dataframe_test['site'] = 'NULL'
dataframe_test['tile'] = 'NULL'
dataframe_test['user_id'] = 'NULL'
dataframe_test['device_id'] = [oldname_map_id[x] for x in pd.read_csv(work_path + '/' + test_split_name + '_device_21Aug17.csv', sep=',', header=None).iloc[:,0]] 

tmp = pd.read_csv(work_path + '/' + test_split_name + '_date_21Aug17.csv', sep=',', header=None)
tmp.columns = ['epoch']
dataframe_test = pd.concat([dataframe_test, tmp], axis=1)

tmp = pd.read_csv(work_path + '/' + test_split_name + '_rss_21Aug17.csv', sep=',', header=None)
tmp[[x for x in tmp.columns]] = tmp[[x for x in tmp.columns]].replace(100, 'NULL')
tmp.columns = ['WAP000'[:-len(str(x))] + str(x) for x in tmp.columns]

dataframe_test = pd.concat([tmp, dataframe_test], axis=1)

dataframe_test['set'] = 'test'


dataframe = pd.concat([dataframe, dataframe_test], axis=0)
dataframe.reset_index(inplace=True, drop=True)
dataframe.reset_index(inplace=True)
dataframe.columns = ['fingerprint_id'] + list(dataframe.columns)[1:]

from datetime import datetime
dataframe['epoch'] = [str(int(datetime.strptime(x, '%Y-%m-%d %H:%M:%S').timestamp())) for x in dataframe['epoch']]



    

dataframe

,device_id,old_name,device_model,device_manufacturer,device_type,notes
0,0,HUAWEI T1 7.0,T1 7.0,Huawei,smartphone,NULL
1,1,Letv x600,x600,LeTV,smartphone,NULL
2,2,Xiaomi MI 4LTE,MI 4LTE,Xiaomi,smartphone,NULL
3,3,Xiaomi MI MAX 2,MI MAX 2,Xiaomi,smartphone,NULL
4,4,samsung SM-A510F,SM-A510F,Samsung,smartphone,NULL
5,5,Meizu MX4 Pro,MX4 Pro,Meizu,smartphone,NULL
6,6,Sony E5823,E5823,Sony,smartphone,NULL
7,7,asus Transformer Prime TF201,Transformer Prime TF201,ASUS,smartphone,NULL
8,9,LGE LG-D625,LG-D625,LGE,smartphone,NULL
9,10,motorola XT1068,XT1068,Motorola,smartphone,NULL


,fingerprint_id,WAP000,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,...,coord_y,coord_z,floor,building,site,tile,user_id,device_id,epoch,set
0,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,19.731,NULL,0,0,NULL,crowd_tile_1,NULL,0,1503050363,training
1,1,NULL,NULL,NULL,NULL,NULL,NULL,NULL,-64,NULL,...,38.706,NULL,3,0,NULL,crowd_tile_1,NULL,1,1490109005,training
2,2,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,48.446,NULL,1,0,NULL,crowd_tile_1,NULL,2,1501765978,training
3,3,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,39.736,NULL,2,0,NULL,crowd_tile_1,NULL,3,1501762670,training
4,4,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,72.874,NULL,1,0,NULL,crowd_tile_1,NULL,4,1503317732,training
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4643,4643,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,16.362,NULL,1,0,NULL,NULL,NULL,12,1503324680,test
4644,4644,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,69.706,NULL,3,0,NULL,NULL,NULL,4,1503318632,test
4645,4645,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,72.311,NULL,3,0,NULL,NULL,NULL,0,1502871017,test
4646,4646,NULL,NULL,NULL,NULL,NULL,NULL,NULL,-57,-39,...,51.000,NULL,2,0,NULL,NULL,NULL,18,1486735275,test


In [10]:
# Generating the file places.csv

places = dataframe[['building', 'floor', 'site']].copy().drop_duplicates()
places.reset_index(inplace=True, drop=True)
places['floor_number'] = places['floor']
places = places[['building', 'floor', 'floor_number', 'site']]
places['site_height'] = 'NULL'
places['site_area'] = 'NULL'
places['floor_height'] = 3.7
places['floor_area'] = 'NULL'
places['building_area'] = 'NULL'

for col in places.columns:
    places[col] = places[col].astype(object)

places.to_csv(work_path + '/places.csv', sep=',', na_rep='NULL', index=False)

places

,building,floor,floor_number,site,site_height,site_area,floor_height,floor_area,building_area
0,0,0,0,NULL,NULL,NULL,3.7,NULL,NULL
1,0,3,3,NULL,NULL,NULL,3.7,NULL,NULL
2,0,1,1,NULL,NULL,NULL,3.7,NULL,NULL
3,0,2,2,NULL,NULL,NULL,3.7,NULL,NULL
4,0,4,4,NULL,NULL,NULL,3.7,NULL,NULL


In [11]:
# Generating the file tessellations.csv

tessellations = dataframe[['building', 'floor', 'site', 'tile']].copy().drop_duplicates()
tessellations['tessellation_type'] = 'crowd'
tessellations['coord_a_x'] = 'NULL'
tessellations['coord_a_y'] = 'NULL'
tessellations['coord_b_x'] = 'NULL'
tessellations['coord_b_y'] = 'NULL'
tessellations['coord_c_x'] = 'NULL'
tessellations['coord_c_y'] = 'NULL'
tessellations['coord_d_x'] = 'NULL'
tessellations['coord_d_y'] = 'NULL'
tessellations = tessellations[(tessellations['tile'] != 'NULL')]


for col in tessellations.columns:
    tessellations[col] = tessellations[col].astype(object)
    

tessellations.to_csv(work_path + '/tessellations.csv', sep=',', na_rep='NULL', index=False)

tessellations

,building,floor,site,tile,tessellation_type,coord_a_x,coord_a_y,coord_b_x,coord_b_y,coord_c_x,coord_c_y,coord_d_x,coord_d_y
0,0,0,NULL,crowd_tile_1,crowd,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1,0,3,NULL,crowd_tile_1,crowd,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2,0,1,NULL,crowd_tile_1,crowd,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
3,0,2,NULL,crowd_tile_1,crowd,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
26,0,4,NULL,crowd_tile_1,crowd,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


In [12]:
# Generating the file fingerprints.csv

fingerprints = dataframe[['fingerprint_id', 'coord_x', 'coord_y', 'coord_z', 'user_id', 'device_id', 'epoch', 'set', 'building', 'floor', 'site', 'tile']].copy()
fingerprints['tile_id'] = fingerprints.merge(tessellations, on=['building', 'floor', 'site', 'tile'], how='left', suffixes=('_1', ''))['tile'].fillna('NULL').values
fingerprints = fingerprints[['fingerprint_id', 'coord_x', 'coord_y', 'coord_z', 'building', 'floor', 'site', 'tile', 'user_id', 'device_id', 'epoch', 'set']]
fingerprints['user_id'] = [x if x != 0 else 'NULL' for x in fingerprints['user_id']]
fingerprints['is_radio_map'] = [True if x == 'training' else False for x in fingerprints['set']]
fingerprints['preceded_by'] = 'NULL'
fingerprints['followed_by'] = 'NULL'
fingerprints['notes'] = 'NULL'

for col in fingerprints.columns:
    fingerprints[col] = fingerprints[col].astype(object)
    

fingerprints.to_csv(work_path + '/fingerprints.csv', sep=',', na_rep='NULL', index=False)

fingerprints

,fingerprint_id,coord_x,coord_y,coord_z,building,floor,site,tile,user_id,device_id,epoch,set,is_radio_map,preceded_by,followed_by,notes
0,0,137.24,19.731,NULL,0,0,NULL,crowd_tile_1,NULL,0,1503050363,training,True,NULL,NULL,NULL
1,1,65.521,38.706,NULL,0,3,NULL,crowd_tile_1,NULL,1,1490109005,training,True,NULL,NULL,NULL
2,2,144.92,48.446,NULL,0,1,NULL,crowd_tile_1,NULL,2,1501765978,training,True,NULL,NULL,NULL
3,3,105.08,39.736,NULL,0,2,NULL,crowd_tile_1,NULL,3,1501762670,training,True,NULL,NULL,NULL
4,4,133.01,72.874,NULL,0,1,NULL,crowd_tile_1,NULL,4,1503317732,training,True,NULL,NULL,NULL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4643,4643,102.97,16.362,NULL,0,1,NULL,NULL,NULL,12,1503324680,test,False,NULL,NULL,NULL
4644,4644,100.11,69.706,NULL,0,3,NULL,NULL,NULL,4,1503318632,test,False,NULL,NULL,NULL
4645,4645,130.86,72.311,NULL,0,3,NULL,NULL,NULL,0,1502871017,test,False,NULL,NULL,NULL
4646,4646,12.667,51,NULL,0,2,NULL,NULL,NULL,18,1486735275,test,False,NULL,NULL,NULL


In [13]:
# Generating the file wifi_obs.csv

wifi_obs = dataframe[[x for x in dataframe.columns if 'WAP' in x]].copy()
wifi_obs['fingerprint_id'] = dataframe['fingerprint_id']
wifi_obs = wifi_obs[['fingerprint_id'] + [x for x in dataframe.columns if 'WAP' in x]]

for col in wifi_obs.columns:
    wifi_obs[col] = wifi_obs[col].astype(object)
    
wifi_obs.columns = ["AP-" + x + "-NULL" if "WAP" in x else x for x in list(wifi_obs.columns)]

wifi_obs.to_csv(work_path + '/wifi_obs.csv', sep=',', na_rep='NULL', index=False)

wifi_obs

,fingerprint_id,AP-WAP000-NULL,AP-WAP001-NULL,AP-WAP002-NULL,AP-WAP003-NULL,AP-WAP004-NULL,AP-WAP005-NULL,AP-WAP006-NULL,AP-WAP007-NULL,AP-WAP008-NULL,...,AP-WAP982-NULL,AP-WAP983-NULL,AP-WAP984-NULL,AP-WAP985-NULL,AP-WAP986-NULL,AP-WAP987-NULL,AP-WAP988-NULL,AP-WAP989-NULL,AP-WAP990-NULL,AP-WAP991-NULL
0,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1,1,NULL,NULL,NULL,NULL,NULL,NULL,NULL,-64,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2,2,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
3,3,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
4,4,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4643,4643,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
4644,4644,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
4645,4645,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
4646,4646,NULL,NULL,NULL,NULL,NULL,NULL,NULL,-57,-39,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


In [ ]:
#                                                                                           #                              
#                                                                                           #   
####################################### PART FOR TUT1 ####################################### 
#                                                                                           #   
#                                                                                           #   

In [14]:
dataset_name = 'TUT1' 


work_path = 'raw_db/' + dataset_name + '/'

file = open(work_path + '/dataset.pickle', 'rb')
dataframe = pickle.load(file)
file.close()


dataframe['coord_z'] = 'NULL'
dataframe['building_id'] = '0'
dataframe['site'] = 'NULL'
dataframe['tile'] = 'NULL'
dataframe['user_id'] = 1
dataframe['device_id'] = 1
dataframe['epoch'] = 'NULL'

dataframe[[x for x in dataframe.columns]] = dataframe[[x for x in dataframe.columns]].replace(100, 'NULL')

dataframe = dataframe[[x for x in dataframe.columns if 'WAP' in x] + ['coord_x', 'coord_y', 'coord_z', 'floor_id', 'building_id', 'site', 'tile', 'user_id', 'device_id', 'epoch', 'set']]
dataframe.reset_index(inplace=True)

dataframe.columns = ['fingerprint_id'] + [x for x in dataframe.columns if 'WAP' in x] + ['coord_x', 'coord_y', 'coord_z', 'floor', 'building', 'site', 'tile', 'user_id', 'device_id', 'epoch', 'set']

#dataframe['tile'] = ['crowd_tile_1' if x == 'training' else 'NULL' for x in dataframe['set']]

# Assigning the grid tiles to the training set fingerprints

def get_tile_id(row):
    if row['set'] == 'training':
        return str(int(math.floor(row['coord_x']))) + "_" + str(int(math.floor(row['coord_y'])))
    else:
        return 'NULL'

dataframe['tile'] = dataframe.apply(lambda row: get_tile_id(row), axis=1)

    
        
dataframe

,fingerprint_id,WAP000,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,...,coord_y,coord_z,floor,building,site,tile,user_id,device_id,epoch,set
0,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,30.50,NULL,0,0,NULL,-75_30,1,1,NULL,training
1,1,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,31.50,NULL,0,0,NULL,-73_31,1,1,NULL,training
2,2,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,29.50,NULL,0,0,NULL,-71_29,1,1,NULL,training
3,3,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,32.50,NULL,0,0,NULL,-71_32,1,1,NULL,training
4,4,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,35.50,NULL,0,0,NULL,-71_35,1,1,NULL,training
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1961,1961,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,6.73,NULL,3,0,NULL,NULL,1,1,NULL,test
1962,1962,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,6.73,NULL,3,0,NULL,NULL,1,1,NULL,test
1963,1963,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,6.73,NULL,3,0,NULL,NULL,1,1,NULL,test
1964,1964,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,6.73,NULL,3,0,NULL,NULL,1,1,NULL,test


In [16]:
# Generating the file places.csv

places = dataframe[['building', 'floor', 'site']].copy().drop_duplicates()
places.reset_index(inplace=True, drop=True)
places['floor_number'] = places['floor']
places = places[['building', 'floor', 'floor_number', 'site']]
places['site_height'] = 'NULL'
places['site_area'] = 'NULL'
places['floor_height'] = 3.7
places['floor_area'] = 'NULL'
places['building_area'] = 'NULL'

for col in places.columns:
    places[col] = places[col].astype(object)

places.to_csv(work_path + '/places.csv', sep=',', na_rep='NULL', index=False)

places

,building,floor,floor_number,site,site_height,site_area,floor_height,floor_area,building_area
0,0,0,0,NULL,NULL,NULL,3.7,NULL,NULL
1,0,1,1,NULL,NULL,NULL,3.7,NULL,NULL
2,0,2,2,NULL,NULL,NULL,3.7,NULL,NULL
3,0,3,3,NULL,NULL,NULL,3.7,NULL,NULL


In [17]:
# Generating the file tessellations.csv


def generate_tile_coords(row, coord_index):
    
    if row['tile'] != 'NULL':
    
        tile_x =  int(row['tile'].split('_')[0])
        tile_y =  int(row['tile'].split('_')[1])

        ax = tile_x 
        ay = tile_y 

        bx = tile_x + 1
        by = tile_y 

        cx = tile_x + 1
        cy = tile_y + 1

        dx = tile_x 
        dy = tile_y + 1
        
        coords = [ax, ay, bx, by, cx, cy, dx, dy]

        return coords[coord_index]
    
    else:
        
        return 'NULL'
    
    
min_x = math.floor(dataframe['coord_x'].min())
max_x = math.floor(dataframe['coord_x'].max())
min_y = math.floor(dataframe['coord_y'].min())
max_y = math.floor(dataframe['coord_y'].max())


grid_tiles = []
for x in range(int(min_x), int(max_x)+1):
    for y in range(int(min_y), int(max_y)+1):
        grid_tiles.append(str(x) + "_" + str(y))


distinct_floors = places[['building', 'floor']].drop_duplicates()
        
tessellations = pd.DataFrame(columns=['tile', 'building', 'floor', 'site'])
for i, row in distinct_floors.iterrows():
    tmp = pd.DataFrame(grid_tiles, columns=['tile'])
    tmp['building'] = row.building
    tmp['floor'] = row.floor
    tmp['site'] = 'NULL'
    tessellations = pd.concat([tessellations, tmp], axis=0)
    
    
tessellations['tessellation_type'] = 'grid'


tessellations['coord_a_x'] = tessellations.apply(lambda row: generate_tile_coords(row, 0), axis=1)
tessellations['coord_a_y'] = tessellations.apply(lambda row: generate_tile_coords(row, 1), axis=1)
tessellations['coord_b_x'] = tessellations.apply(lambda row: generate_tile_coords(row, 2), axis=1)
tessellations['coord_b_y'] = tessellations.apply(lambda row: generate_tile_coords(row, 3), axis=1)
tessellations['coord_c_x'] = tessellations.apply(lambda row: generate_tile_coords(row, 4), axis=1)
tessellations['coord_c_y'] = tessellations.apply(lambda row: generate_tile_coords(row, 5), axis=1)
tessellations['coord_d_x'] = tessellations.apply(lambda row: generate_tile_coords(row, 6), axis=1)
tessellations['coord_d_y'] = tessellations.apply(lambda row: generate_tile_coords(row, 7), axis=1)

    
tessellations = tessellations[['building','floor','site','tile','tessellation_type','coord_a_x','coord_a_y','coord_b_x','coord_b_y','coord_c_x','coord_c_y','coord_d_x','coord_d_y']]

for col in tessellations.columns:
    tessellations[col] = tessellations[col].astype(object)
    

tessellations.to_csv(work_path + '/tessellations.csv', sep=',', na_rep='NULL', index=False)

tessellations

,building,floor,site,tile,tessellation_type,coord_a_x,coord_a_y,coord_b_x,coord_b_y,coord_c_x,coord_c_y,coord_d_x,coord_d_y
0,0,0,NULL,-105_-23,grid,-105,-23,-104,-23,-104,-22,-105,-22
1,0,0,NULL,-105_-22,grid,-105,-22,-104,-22,-104,-21,-105,-21
2,0,0,NULL,-105_-21,grid,-105,-21,-104,-21,-104,-20,-105,-20
3,0,0,NULL,-105_-20,grid,-105,-20,-104,-20,-104,-19,-105,-19
4,0,0,NULL,-105_-19,grid,-105,-19,-104,-19,-104,-18,-105,-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9671,0,3,NULL,58_31,grid,58,31,59,31,59,32,58,32
9672,0,3,NULL,58_32,grid,58,32,59,32,59,33,58,33
9673,0,3,NULL,58_33,grid,58,33,59,33,59,34,58,34
9674,0,3,NULL,58_34,grid,58,34,59,34,59,35,58,35


In [20]:
# Generating the file adjacences.csv

# tile_1_building, tile_1_floor, tile_1_site, tile_1_tile, tile_2_building, tile_2_floor, tile_2_site, tile_2_tile, walkable, cost

all_tiles = tessellations[['building', 'floor', 'site','tile']].drop_duplicates().values

adjacences = []
for row in all_tiles:
    tile = row[3]
    x_cell = int(tile.split('_')[0])
    y_cell = int(tile.split('_')[1])
    
    neighbours = []
    for mod1 in [-1, 0, 1]:
        for mod2 in [-1, 0, 1]:
            neighbours.append((x_cell + mod1, y_cell + mod2))
  
    for neigh in neighbours: 
        if not(neigh[0] == x_cell and neigh[1] == y_cell) and neigh[0] >= int(min_x) and neigh[1] >= int(min_y) and neigh[0] <= int(max_x) and neigh[1] <= int(max_y):
            adjacences.append([str(row[0]), str(row[1]), 'NULL', tile, str(row[0]), str(row[1]), 'NULL', str(neigh[0]) + "_" + str(neigh[1]), 1, 'NULL'])
            

adjacences = pd.DataFrame(adjacences, columns=['tile_1_building', 'tile_1_floor', 'tile_1_site', 'tile_1_tile', 'tile_2_building', 'tile_2_floor', 'tile_2_site', 'tile_2_tile', 'walkable', 'cost'])

adjacences.to_csv(work_path + '/adjacences.csv', sep=',', na_rep='NULL', index=False)

adjacences

,tile_1_building,tile_1_floor,tile_1_site,tile_1_tile,tile_2_building,tile_2_floor,tile_2_site,tile_2_tile,walkable,cost
0,0,0,NULL,-105_-23,0,0,NULL,-105_-22,1,NULL
1,0,0,NULL,-105_-23,0,0,NULL,-104_-23,1,NULL
2,0,0,NULL,-105_-23,0,0,NULL,-104_-22,1,NULL
3,0,0,NULL,-105_-22,0,0,NULL,-105_-23,1,NULL
4,0,0,NULL,-105_-22,0,0,NULL,-105_-21,1,NULL
...,...,...,...,...,...,...,...,...,...,...
304291,0,3,NULL,58_34,0,3,NULL,58_33,1,NULL
304292,0,3,NULL,58_34,0,3,NULL,58_35,1,NULL
304293,0,3,NULL,58_35,0,3,NULL,57_34,1,NULL
304294,0,3,NULL,58_35,0,3,NULL,57_35,1,NULL


In [21]:
# Generating the file fingerprints.csv

fingerprints = dataframe[['fingerprint_id', 'coord_x', 'coord_y', 'coord_z', 'user_id', 'device_id', 'epoch', 'set', 'building', 'floor', 'site', 'tile']].copy()
fingerprints['tile_id'] = fingerprints.merge(tessellations, on=['building', 'floor', 'site', 'tile'], how='left', suffixes=('_1', ''))['tile'].fillna('NULL').values
fingerprints = fingerprints[['fingerprint_id', 'coord_x', 'coord_y', 'coord_z', 'building', 'floor', 'site', 'tile', 'user_id', 'device_id', 'epoch', 'set']]
fingerprints['user_id'] = [x if x != 0 else 'NULL' for x in fingerprints['user_id']]
fingerprints['is_radio_map'] = [True if x == 'training' else False for x in fingerprints['set']]
fingerprints['preceded_by'] = 'NULL'
fingerprints['followed_by'] = 'NULL'
fingerprints['notes'] = 'NULL'

for col in fingerprints.columns:
    fingerprints[col] = fingerprints[col].astype(object)
    

fingerprints.to_csv(work_path + '/fingerprints.csv', sep=',', na_rep='NULL', index=False)

fingerprints

,fingerprint_id,coord_x,coord_y,coord_z,building,floor,site,tile,user_id,device_id,epoch,set,is_radio_map,preceded_by,followed_by,notes
0,0,-74.5,30.5,NULL,0,0,NULL,-75_30,1,1,NULL,training,True,NULL,NULL,NULL
1,1,-72.5,31.5,NULL,0,0,NULL,-73_31,1,1,NULL,training,True,NULL,NULL,NULL
2,2,-70.5,29.5,NULL,0,0,NULL,-71_29,1,1,NULL,training,True,NULL,NULL,NULL
3,3,-70.5,32.5,NULL,0,0,NULL,-71_32,1,1,NULL,training,True,NULL,NULL,NULL
4,4,-70.5,35.5,NULL,0,0,NULL,-71_35,1,1,NULL,training,True,NULL,NULL,NULL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1961,1961,1.44,6.73,NULL,0,3,NULL,NULL,1,1,NULL,test,False,NULL,NULL,NULL
1962,1962,1.44,6.73,NULL,0,3,NULL,NULL,1,1,NULL,test,False,NULL,NULL,NULL
1963,1963,1.44,6.73,NULL,0,3,NULL,NULL,1,1,NULL,test,False,NULL,NULL,NULL
1964,1964,1.44,6.73,NULL,0,3,NULL,NULL,1,1,NULL,test,False,NULL,NULL,NULL


In [22]:
# Generating the file wifi_obs.csv

wifi_obs = dataframe[[x for x in dataframe.columns if 'WAP' in x]].copy()
wifi_obs['fingerprint_id'] = dataframe['fingerprint_id']
wifi_obs = wifi_obs[['fingerprint_id'] + [x for x in dataframe.columns if 'WAP' in x]]

for col in wifi_obs.columns:
    wifi_obs[col] = wifi_obs[col].astype(object)
    
wifi_obs.columns = ["AP-" + x + "-NULL" if "WAP" in x else x for x in list(wifi_obs.columns)]

wifi_obs.to_csv(work_path + '/wifi_obs.csv', sep=',', na_rep='NULL', index=False)

wifi_obs

,fingerprint_id,AP-WAP000-NULL,AP-WAP001-NULL,AP-WAP002-NULL,AP-WAP003-NULL,AP-WAP004-NULL,AP-WAP005-NULL,AP-WAP006-NULL,AP-WAP007-NULL,AP-WAP008-NULL,...,AP-WAP299-NULL,AP-WAP300-NULL,AP-WAP301-NULL,AP-WAP302-NULL,AP-WAP303-NULL,AP-WAP304-NULL,AP-WAP305-NULL,AP-WAP306-NULL,AP-WAP307-NULL,AP-WAP308-NULL
0,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1,1,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2,2,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
3,3,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
4,4,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1961,1961,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,-82,NULL,NULL
1962,1962,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,-86,NULL,NULL
1963,1963,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,-88,NULL,NULL
1964,1964,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,-89,NULL,NULL


In [ ]:
#                                                                                           #                              
#                                                                                           #   
####################################### PART FOR TUT6 ####################################### 
#                                                                                           #   
#                                                                                           #   

In [23]:
dataset_name = 'TUT6' 


work_path = 'raw_db/' + dataset_name + '/'

file = open(work_path + '/dataset.pickle', 'rb')
dataframe = pickle.load(file)
file.close()


dataframe['coord_z'] = 'NULL'
dataframe['building_id'] = '0'
dataframe['site'] = 'NULL'
dataframe['tile'] = 'NULL'
dataframe['user_id'] = 'NULL'
dataframe['device_id'] = 'NULL'
dataframe['epoch'] = 'NULL'

dataframe[[x for x in dataframe.columns]] = dataframe[[x for x in dataframe.columns]].replace(100, 'NULL')

dataframe = dataframe[[x for x in dataframe.columns if 'WAP' in x] + ['coord_x', 'coord_y', 'coord_z', 'floor_id', 'building_id', 'site', 'tile', 'user_id', 'device_id', 'epoch', 'set']]
dataframe.reset_index(inplace=True)

dataframe.columns = ['fingerprint_id'] + [x for x in dataframe.columns if 'WAP' in x] + ['coord_x', 'coord_y', 'coord_z', 'floor', 'building', 'site', 'tile', 'user_id', 'device_id', 'epoch', 'set']

dataframe['tile'] = ['crowd_tile_1' if x == 'training' else 'NULL' for x in dataframe['set']]

        
dataframe

,fingerprint_id,WAP000,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,...,coord_y,coord_z,floor,building,site,tile,user_id,device_id,epoch,set
0,0,NULL,-66,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,52.128,NULL,0,0,NULL,crowd_tile_1,NULL,NULL,NULL,training
1,1,NULL,-63,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,52.939,NULL,0,0,NULL,crowd_tile_1,NULL,NULL,NULL,training
2,2,NULL,-60,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,35.100,NULL,0,0,NULL,crowd_tile_1,NULL,NULL,NULL,training
3,3,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,48.483,NULL,3,0,NULL,crowd_tile_1,NULL,NULL,NULL,training
4,4,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,32.148,NULL,0,0,NULL,crowd_tile_1,NULL,NULL,NULL,training
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10380,10380,NULL,-75,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,22.050,NULL,0,0,NULL,NULL,NULL,NULL,NULL,test
10381,10381,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,44.384,NULL,2,0,NULL,NULL,NULL,NULL,NULL,test
10382,10382,NULL,-78,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,29.741,NULL,0,0,NULL,NULL,NULL,NULL,NULL,test
10383,10383,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,28.627,NULL,0,0,NULL,NULL,NULL,NULL,NULL,test


In [25]:
# Generating the file places.csv

places = dataframe[['building', 'floor', 'site']].copy().drop_duplicates()
places.reset_index(inplace=True, drop=True)
places['floor_number'] = places['floor']
places = places[['building', 'floor', 'floor_number', 'site']]
places['site_height'] = 'NULL'
places['site_area'] = 'NULL'
places['floor_height'] = 3.7
places['floor_area'] = 'NULL'
places['building_area'] = 'NULL'

for col in places.columns:
    places[col] = places[col].astype(object)

places.to_csv(work_path + '/places.csv', sep=',', na_rep='NULL', index=False)

places

,building,floor,floor_number,site,site_height,site_area,floor_height,floor_area,building_area
0,0,0,0,NULL,NULL,NULL,3.7,NULL,NULL
1,0,3,3,NULL,NULL,NULL,3.7,NULL,NULL
2,0,2,2,NULL,NULL,NULL,3.7,NULL,NULL
3,0,1,1,NULL,NULL,NULL,3.7,NULL,NULL


In [26]:
# Generating the file tessellations.csv

tessellations = dataframe[['building', 'floor', 'site', 'tile']].copy().drop_duplicates()
tessellations['tessellation_type'] = 'crowd'
tessellations['coord_a_x'] = 'NULL'
tessellations['coord_a_y'] = 'NULL'
tessellations['coord_b_x'] = 'NULL'
tessellations['coord_b_y'] = 'NULL'
tessellations['coord_c_x'] = 'NULL'
tessellations['coord_c_y'] = 'NULL'
tessellations['coord_d_x'] = 'NULL'
tessellations['coord_d_y'] = 'NULL'
tessellations = tessellations[(tessellations['tile'] != 'NULL')]


for col in tessellations.columns:
    tessellations[col] = tessellations[col].astype(object)
    

tessellations.to_csv(work_path + '/tessellations.csv', sep=',', na_rep='NULL', index=False)

tessellations

,building,floor,site,tile,tessellation_type,coord_a_x,coord_a_y,coord_b_x,coord_b_y,coord_c_x,coord_c_y,coord_d_x,coord_d_y
0,0,0,NULL,crowd_tile_1,crowd,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
3,0,3,NULL,crowd_tile_1,crowd,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
7,0,2,NULL,crowd_tile_1,crowd,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
9,0,1,NULL,crowd_tile_1,crowd,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


In [27]:
# Generating the file fingerprints.csv

fingerprints = dataframe[['fingerprint_id', 'coord_x', 'coord_y', 'coord_z', 'user_id', 'device_id', 'epoch', 'set', 'building', 'floor', 'site', 'tile']].copy()
fingerprints['tile_id'] = fingerprints.merge(tessellations, on=['building', 'floor', 'site', 'tile'], how='left', suffixes=('_1', ''))['tile'].fillna('NULL').values
fingerprints = fingerprints[['fingerprint_id', 'coord_x', 'coord_y', 'coord_z', 'building', 'floor', 'site', 'tile', 'user_id', 'device_id', 'epoch', 'set']]
fingerprints['user_id'] = [x if x != 0 else 'NULL' for x in fingerprints['user_id']]
fingerprints['is_radio_map'] = [True if x == 'training' else False for x in fingerprints['set']]
fingerprints['preceded_by'] = 'NULL'
fingerprints['followed_by'] = 'NULL'
fingerprints['notes'] = 'NULL'

for col in fingerprints.columns:
    fingerprints[col] = fingerprints[col].astype(object)
    

fingerprints.to_csv(work_path + '/fingerprints.csv', sep=',', na_rep='NULL', index=False)

fingerprints

,fingerprint_id,coord_x,coord_y,coord_z,building,floor,site,tile,user_id,device_id,epoch,set,is_radio_map,preceded_by,followed_by,notes
0,0,129.19,52.128,NULL,0,0,NULL,crowd_tile_1,NULL,NULL,NULL,training,True,NULL,NULL,NULL
1,1,174.57,52.939,NULL,0,0,NULL,crowd_tile_1,NULL,NULL,NULL,training,True,NULL,NULL,NULL
2,2,133.23,35.1,NULL,0,0,NULL,crowd_tile_1,NULL,NULL,NULL,training,True,NULL,NULL,NULL
3,3,133.93,48.483,NULL,0,3,NULL,crowd_tile_1,NULL,NULL,NULL,training,True,NULL,NULL,NULL
4,4,85.708,32.148,NULL,0,0,NULL,crowd_tile_1,NULL,NULL,NULL,training,True,NULL,NULL,NULL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10380,10380,153.17,22.05,NULL,0,0,NULL,NULL,NULL,NULL,NULL,test,False,NULL,NULL,NULL
10381,10381,101.78,44.384,NULL,0,2,NULL,NULL,NULL,NULL,NULL,test,False,NULL,NULL,NULL
10382,10382,114.59,29.741,NULL,0,0,NULL,NULL,NULL,NULL,NULL,test,False,NULL,NULL,NULL
10383,10383,62.857,28.627,NULL,0,0,NULL,NULL,NULL,NULL,NULL,test,False,NULL,NULL,NULL


In [28]:
# Generating the file wifi_obs.csv

wifi_obs = dataframe[[x for x in dataframe.columns if 'WAP' in x]].copy()
wifi_obs['fingerprint_id'] = dataframe['fingerprint_id']
wifi_obs = wifi_obs[['fingerprint_id'] + [x for x in dataframe.columns if 'WAP' in x]]

for col in wifi_obs.columns:
    wifi_obs[col] = wifi_obs[col].astype(object)
    
wifi_obs.columns = ["AP-" + x + "-NULL" if "WAP" in x else x for x in list(wifi_obs.columns)]

wifi_obs.to_csv(work_path + '/wifi_obs.csv', sep=',', na_rep='NULL', index=False)

wifi_obs

,fingerprint_id,AP-WAP000-NULL,AP-WAP001-NULL,AP-WAP002-NULL,AP-WAP003-NULL,AP-WAP004-NULL,AP-WAP005-NULL,AP-WAP006-NULL,AP-WAP007-NULL,AP-WAP008-NULL,...,AP-WAP642-NULL,AP-WAP643-NULL,AP-WAP644-NULL,AP-WAP645-NULL,AP-WAP646-NULL,AP-WAP647-NULL,AP-WAP648-NULL,AP-WAP649-NULL,AP-WAP650-NULL,AP-WAP651-NULL
0,0,NULL,-66,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1,1,NULL,-63,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2,2,NULL,-60,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
3,3,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
4,4,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10380,10380,NULL,-75,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
10381,10381,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
10382,10382,NULL,-78,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
10383,10383,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


In [ ]:
#                                                                                           #                              
#                                                                                           #   
####################################### PART FOR TUT2 ####################################### 
#                                                                                           #   
#                                                                                           #   

In [29]:
dataset_name = 'TUT2' 


work_path = 'raw_db/' + dataset_name + '/'

file = open(work_path + '/dataset.pickle', 'rb')
dataframe = pickle.load(file)
file.close()


dataframe['coord_z'] = 'NULL'
dataframe['building_id'] = '1'
dataframe['site'] = 'NULL'
dataframe['tile'] = 'NULL'
dataframe['user_id'] = 1
dataframe['device_id'] = 1
dataframe['epoch'] = 'NULL'

dataframe[[x for x in dataframe.columns]] = dataframe[[x for x in dataframe.columns]].replace(100, 'NULL')

dataframe = dataframe[[x for x in dataframe.columns if 'WAP' in x] + ['coord_x', 'coord_y', 'coord_z', 'floor_id', 'building_id', 'site', 'tile', 'user_id', 'device_id', 'epoch', 'set']]
dataframe.reset_index(inplace=True)

dataframe.columns = ['fingerprint_id'] + [x for x in dataframe.columns if 'WAP' in x] + ['coord_x', 'coord_y', 'coord_z', 'floor', 'building', 'site', 'tile', 'user_id', 'device_id', 'epoch', 'set']

#dataframe['tile'] = ['crowd_tile_1' if x == 'training' else 'NULL' for x in dataframe['set']]

# Assigning the grid tiles to the training set fingerprints

def get_tile_id(row):
    if row['set'] == 'training':
        return str(int(math.floor(row['coord_x']))) + "_" + str(int(math.floor(row['coord_y'])))
    else:
        return 'NULL'

dataframe['tile'] = dataframe.apply(lambda row: get_tile_id(row), axis=1)

    
        
dataframe

,fingerprint_id,WAP000,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,...,coord_y,coord_z,floor,building,site,tile,user_id,device_id,epoch,set
0,0,NULL,NULL,NULL,NULL,NULL,-62,-62,-61,-62,...,-0.50,NULL,0,1,NULL,2_-1,1,1,NULL,training
1,1,NULL,NULL,NULL,NULL,NULL,-70,-69,-70,-69,...,-3.50,NULL,0,1,NULL,2_-4,1,1,NULL,training
2,2,NULL,NULL,NULL,NULL,NULL,-70,-69,-70,-69,...,-10.50,NULL,0,1,NULL,3_-11,1,1,NULL,training
3,3,NULL,NULL,NULL,NULL,NULL,-70,-69,-70,-69,...,-15.50,NULL,0,1,NULL,3_-16,1,1,NULL,training
4,4,NULL,NULL,NULL,NULL,NULL,-70,-78,-70,-82,...,-21.50,NULL,0,1,NULL,3_-22,1,1,NULL,training
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,755,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,43.97,NULL,1,1,NULL,NULL,1,1,NULL,test
756,756,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,44.28,NULL,1,1,NULL,NULL,1,1,NULL,test
757,757,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,43.88,NULL,1,1,NULL,NULL,1,1,NULL,test
758,758,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,44.10,NULL,1,1,NULL,NULL,1,1,NULL,test


In [33]:
# Generating the file places.csv

places = dataframe[['building', 'floor', 'site']].copy().drop_duplicates()
places.reset_index(inplace=True, drop=True)
places['floor_number'] = places['floor']
places = places[['building', 'floor', 'floor_number', 'site']]
places['site_height'] = 'NULL'
places['site_area'] = 'NULL'
places['floor_height'] = 3.5
places['floor_area'] = 'NULL'
places['building_area'] = 'NULL'

for col in places.columns:
    places[col] = places[col].astype(object)

places.to_csv(work_path + '/places.csv', sep=',', na_rep='NULL', index=False)

places

,building,floor,floor_number,site,site_height,site_area,floor_height,floor_area,building_area
0,1,0,0,NULL,NULL,NULL,3.5,NULL,NULL
1,1,1,1,NULL,NULL,NULL,3.5,NULL,NULL
2,1,2,2,NULL,NULL,NULL,3.5,NULL,NULL


In [34]:
# Generating the file tessellations.csv


def generate_tile_coords(row, coord_index):
    
    if row['tile'] != 'NULL':
    
        tile_x =  int(row['tile'].split('_')[0])
        tile_y =  int(row['tile'].split('_')[1])

        ax = tile_x 
        ay = tile_y 

        bx = tile_x + 1
        by = tile_y 

        cx = tile_x + 1
        cy = tile_y + 1

        dx = tile_x 
        dy = tile_y + 1
        
        coords = [ax, ay, bx, by, cx, cy, dx, dy]

        return coords[coord_index]
    
    else:
        
        return 'NULL'
    
    
min_x = math.floor(dataframe['coord_x'].min())
max_x = math.floor(dataframe['coord_x'].max())
min_y = math.floor(dataframe['coord_y'].min())
max_y = math.floor(dataframe['coord_y'].max())


grid_tiles = []
for x in range(int(min_x), int(max_x)+1):
    for y in range(int(min_y), int(max_y)+1):
        grid_tiles.append(str(x) + "_" + str(y))


distinct_floors = places[['building', 'floor']].drop_duplicates()
        
tessellations = pd.DataFrame(columns=['tile', 'building', 'floor', 'site'])
for i, row in distinct_floors.iterrows():
    tmp = pd.DataFrame(grid_tiles, columns=['tile'])
    tmp['building'] = row.building
    tmp['floor'] = row.floor
    tmp['site'] = 'NULL'
    tessellations = pd.concat([tessellations, tmp], axis=0)
    
    
tessellations['tessellation_type'] = 'grid'


tessellations['coord_a_x'] = tessellations.apply(lambda row: generate_tile_coords(row, 0), axis=1)
tessellations['coord_a_y'] = tessellations.apply(lambda row: generate_tile_coords(row, 1), axis=1)
tessellations['coord_b_x'] = tessellations.apply(lambda row: generate_tile_coords(row, 2), axis=1)
tessellations['coord_b_y'] = tessellations.apply(lambda row: generate_tile_coords(row, 3), axis=1)
tessellations['coord_c_x'] = tessellations.apply(lambda row: generate_tile_coords(row, 4), axis=1)
tessellations['coord_c_y'] = tessellations.apply(lambda row: generate_tile_coords(row, 5), axis=1)
tessellations['coord_d_x'] = tessellations.apply(lambda row: generate_tile_coords(row, 6), axis=1)
tessellations['coord_d_y'] = tessellations.apply(lambda row: generate_tile_coords(row, 7), axis=1)

    
tessellations = tessellations[['building','floor','site','tile','tessellation_type','coord_a_x','coord_a_y','coord_b_x','coord_b_y','coord_c_x','coord_c_y','coord_d_x','coord_d_y']]

for col in tessellations.columns:
    tessellations[col] = tessellations[col].astype(object)
    

tessellations.to_csv(work_path + '/tessellations.csv', sep=',', na_rep='NULL', index=False)

tessellations

,building,floor,site,tile,tessellation_type,coord_a_x,coord_a_y,coord_b_x,coord_b_y,coord_c_x,coord_c_y,coord_d_x,coord_d_y
0,1,0,NULL,-123_-43,grid,-123,-43,-122,-43,-122,-42,-123,-42
1,1,0,NULL,-123_-42,grid,-123,-42,-122,-42,-122,-41,-123,-41
2,1,0,NULL,-123_-41,grid,-123,-41,-122,-41,-122,-40,-123,-40
3,1,0,NULL,-123_-40,grid,-123,-40,-122,-40,-122,-39,-123,-39
4,1,0,NULL,-123_-39,grid,-123,-39,-122,-39,-122,-38,-123,-38
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14377,1,2,NULL,29_46,grid,29,46,30,46,30,47,29,47
14378,1,2,NULL,29_47,grid,29,47,30,47,30,48,29,48
14379,1,2,NULL,29_48,grid,29,48,30,48,30,49,29,49
14380,1,2,NULL,29_49,grid,29,49,30,49,30,50,29,50


In [37]:
# Generating the file adjacences.csv

# tile_1_building, tile_1_floor, tile_1_site, tile_1_tile, tile_2_building, tile_2_floor, tile_2_site, tile_2_tile, walkable, cost

all_tiles = tessellations[['building', 'floor', 'site','tile']].drop_duplicates().values

adjacences = []
for row in all_tiles:
    tile = row[3]
    x_cell = int(tile.split('_')[0])
    y_cell = int(tile.split('_')[1])
    
    neighbours = []
    for mod1 in [-1, 0, 1]:
        for mod2 in [-1, 0, 1]:
            neighbours.append((x_cell + mod1, y_cell + mod2))
  
    for neigh in neighbours: 
        if not(neigh[0] == x_cell and neigh[1] == y_cell) and neigh[0] >= int(min_x) and neigh[1] >= int(min_y) and neigh[0] <= int(max_x) and neigh[1] <= int(max_y):
            adjacences.append([str(row[0]), str(row[1]), 'NULL', tile, str(row[0]), str(row[1]), 'NULL', str(neigh[0]) + "_" + str(neigh[1]), 1, 'NULL'])
            

adjacences = pd.DataFrame(adjacences, columns=['tile_1_building', 'tile_1_floor', 'tile_1_site', 'tile_1_tile', 'tile_2_building', 'tile_2_floor', 'tile_2_site', 'tile_2_tile', 'walkable', 'cost'])

adjacences.to_csv(work_path + '/adjacences.csv', sep=',', na_rep='NULL', index=False)

adjacences

,tile_1_building,tile_1_floor,tile_1_site,tile_1_tile,tile_2_building,tile_2_floor,tile_2_site,tile_2_tile,walkable,cost
0,1,0,NULL,-123_-43,1,0,NULL,-123_-42,1,NULL
1,1,0,NULL,-123_-43,1,0,NULL,-122_-43,1,NULL
2,1,0,NULL,-123_-43,1,0,NULL,-122_-42,1,NULL
3,1,0,NULL,-123_-42,1,0,NULL,-123_-43,1,NULL
4,1,0,NULL,-123_-42,1,0,NULL,-123_-41,1,NULL
...,...,...,...,...,...,...,...,...,...,...
340729,1,2,NULL,29_49,1,2,NULL,29_48,1,NULL
340730,1,2,NULL,29_49,1,2,NULL,29_50,1,NULL
340731,1,2,NULL,29_50,1,2,NULL,28_49,1,NULL
340732,1,2,NULL,29_50,1,2,NULL,28_50,1,NULL


In [39]:
# Generating the file fingerprints.csv

fingerprints = dataframe[['fingerprint_id', 'coord_x', 'coord_y', 'coord_z', 'user_id', 'device_id', 'epoch', 'set', 'building', 'floor', 'site', 'tile']].copy()
fingerprints['tile_id'] = fingerprints.merge(tessellations, on=['building', 'floor', 'site', 'tile'], how='left', suffixes=('_1', ''))['tile'].fillna('NULL').values
fingerprints = fingerprints[['fingerprint_id', 'coord_x', 'coord_y', 'coord_z', 'building', 'floor', 'site', 'tile', 'user_id', 'device_id', 'epoch', 'set']]
fingerprints['user_id'] = [x if x != 0 else 'NULL' for x in fingerprints['user_id']]
fingerprints['is_radio_map'] = [True if x == 'training' else False for x in fingerprints['set']]
fingerprints['preceded_by'] = 'NULL'
fingerprints['followed_by'] = 'NULL'
fingerprints['notes'] = 'NULL'

for col in fingerprints.columns:
    fingerprints[col] = fingerprints[col].astype(object)
    

fingerprints.to_csv(work_path + '/fingerprints.csv', sep=',', na_rep='NULL', index=False)

fingerprints

,fingerprint_id,coord_x,coord_y,coord_z,building,floor,site,tile,user_id,device_id,epoch,set,is_radio_map,preceded_by,followed_by,notes
0,0,2.5,-0.5,NULL,1,0,NULL,2_-1,1,1,NULL,training,True,NULL,NULL,NULL
1,1,2.5,-3.5,NULL,1,0,NULL,2_-4,1,1,NULL,training,True,NULL,NULL,NULL
2,2,3.5,-10.5,NULL,1,0,NULL,3_-11,1,1,NULL,training,True,NULL,NULL,NULL
3,3,3.5,-15.5,NULL,1,0,NULL,3_-16,1,1,NULL,training,True,NULL,NULL,NULL
4,4,3.5,-21.5,NULL,1,0,NULL,3_-22,1,1,NULL,training,True,NULL,NULL,NULL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,755,-13.42,43.97,NULL,1,1,NULL,NULL,1,1,NULL,test,False,NULL,NULL,NULL
756,756,-9.98,44.28,NULL,1,1,NULL,NULL,1,1,NULL,test,False,NULL,NULL,NULL
757,757,-5.73,43.88,NULL,1,1,NULL,NULL,1,1,NULL,test,False,NULL,NULL,NULL
758,758,-1.12,44.1,NULL,1,1,NULL,NULL,1,1,NULL,test,False,NULL,NULL,NULL


In [40]:
# Generating the file wifi_obs.csv

wifi_obs = dataframe[[x for x in dataframe.columns if 'WAP' in x]].copy()
wifi_obs['fingerprint_id'] = dataframe['fingerprint_id']
wifi_obs = wifi_obs[['fingerprint_id'] + [x for x in dataframe.columns if 'WAP' in x]]

for col in wifi_obs.columns:
    wifi_obs[col] = wifi_obs[col].astype(object)
    
wifi_obs.columns = ["AP-" + x + "-NULL" if "WAP" in x else x for x in list(wifi_obs.columns)]

wifi_obs.to_csv(work_path + '/wifi_obs.csv', sep=',', na_rep='NULL', index=False)

wifi_obs

,fingerprint_id,AP-WAP000-NULL,AP-WAP001-NULL,AP-WAP002-NULL,AP-WAP003-NULL,AP-WAP004-NULL,AP-WAP005-NULL,AP-WAP006-NULL,AP-WAP007-NULL,AP-WAP008-NULL,...,AP-WAP344-NULL,AP-WAP345-NULL,AP-WAP346-NULL,AP-WAP347-NULL,AP-WAP348-NULL,AP-WAP349-NULL,AP-WAP350-NULL,AP-WAP351-NULL,AP-WAP352-NULL,AP-WAP353-NULL
0,0,NULL,NULL,NULL,NULL,NULL,-62,-62,-61,-62,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1,1,NULL,NULL,NULL,NULL,NULL,-70,-69,-70,-69,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2,2,NULL,NULL,NULL,NULL,NULL,-70,-69,-70,-69,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
3,3,NULL,NULL,NULL,NULL,NULL,-70,-69,-70,-69,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
4,4,NULL,NULL,NULL,NULL,NULL,-70,-78,-70,-82,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,755,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
756,756,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
757,757,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
758,758,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


In [ ]:
#                                                                                           #                              
#                                                                                           #   
####################################### PART FOR TUT5 ####################################### 
#                                                                                           #   
#                                                                                           #   

In [41]:
dataset_name = 'TUT5' 


work_path = 'raw_db/' + dataset_name + '/'

file = open(work_path + '/dataset.pickle', 'rb')
dataframe = pickle.load(file)
file.close()


dataframe['coord_z'] = 'NULL'
dataframe['building_id'] = '1'
dataframe['site'] = 'NULL'
dataframe['tile'] = 'NULL'
dataframe['user_id'] = 'NULL'
dataframe['device_id'] = 'NULL'
dataframe['epoch'] = 'NULL'

dataframe[[x for x in dataframe.columns]] = dataframe[[x for x in dataframe.columns]].replace(100, 'NULL')

dataframe = dataframe[[x for x in dataframe.columns if 'WAP' in x] + ['coord_x', 'coord_y', 'coord_z', 'floor_id', 'building_id', 'site', 'tile', 'user_id', 'device_id', 'epoch', 'set']]
dataframe.reset_index(inplace=True)

dataframe.columns = ['fingerprint_id'] + [x for x in dataframe.columns if 'WAP' in x] + ['coord_x', 'coord_y', 'coord_z', 'floor', 'building', 'site', 'tile', 'user_id', 'device_id', 'epoch', 'set']

#dataframe['tile'] = ['crowd_tile_1' if x == 'training' else 'NULL' for x in dataframe['set']]

# Assigning the grid tiles to the training set fingerprints

def get_tile_id(row):
    if row['set'] == 'training':
        return str(int(math.floor(row['coord_x']))) + "_" + str(int(math.floor(row['coord_y'])))
    else:
        return 'NULL'

dataframe['tile'] = dataframe.apply(lambda row: get_tile_id(row), axis=1)

    
        
dataframe

,fingerprint_id,WAP000,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,...,coord_y,coord_z,floor,building,site,tile,user_id,device_id,epoch,set
0,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,72.043,NULL,0,1,NULL,172_72,NULL,NULL,NULL,training
1,1,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,77.043,NULL,0,1,NULL,172_77,NULL,NULL,NULL,training
2,2,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,77.043,NULL,0,1,NULL,167_77,NULL,NULL,NULL,training
3,3,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,82.043,NULL,0,1,NULL,167_82,NULL,NULL,NULL,training
4,4,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,87.043,NULL,0,1,NULL,167_87,NULL,NULL,NULL,training
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,1423,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,115.720,NULL,2,1,NULL,NULL,NULL,NULL,NULL,test
1424,1424,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,113.720,NULL,2,1,NULL,NULL,NULL,NULL,NULL,test
1425,1425,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,111.720,NULL,2,1,NULL,NULL,NULL,NULL,NULL,test
1426,1426,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,109.720,NULL,2,1,NULL,NULL,NULL,NULL,NULL,test


In [50]:
# Generating the file places.csv

places = dataframe[['building', 'floor', 'site']].copy().drop_duplicates()
places.reset_index(inplace=True, drop=True)
places['floor_number'] = places['floor']
places = places[['building', 'floor', 'floor_number', 'site']]
places['site_height'] = 'NULL'
places['site_area'] = 'NULL'
places['floor_height'] = 3.5
places['floor_area'] = 'NULL'
places['building_area'] = 'NULL'

for col in places.columns:
    places[col] = places[col].astype(object)

places.to_csv(work_path + '/places.csv', sep=',', na_rep='NULL', index=False)

places

,building,floor,floor_number,site,site_height,site_area,floor_height,floor_area,building_area
0,1,0,0,NULL,NULL,NULL,3.5,NULL,NULL
1,1,1,1,NULL,NULL,NULL,3.5,NULL,NULL
2,1,2,2,NULL,NULL,NULL,3.5,NULL,NULL


In [52]:
# Generating the file tessellations.csv


def generate_tile_coords(row, coord_index):
    
    if row['tile'] != 'NULL':
    
        tile_x =  int(row['tile'].split('_')[0])
        tile_y =  int(row['tile'].split('_')[1])

        ax = tile_x - 2.5
        ay = tile_y - 2.5

        bx = tile_x + 2.5
        by = tile_y - 2.5

        cx = tile_x + 2.5
        cy = tile_y + 2.5

        dx = tile_x - 2.5
        dy = tile_y + 2.5
        
        coords = [ax, ay, bx, by, cx, cy, dx, dy]

        return coords[coord_index]
    
    else:
        
        return 'NULL'
    
    
min_x = math.floor(dataframe['coord_x'].min())
max_x = math.floor(dataframe['coord_x'].max())
min_y = math.floor(dataframe['coord_y'].min())
max_y = math.floor(dataframe['coord_y'].max())


grid_tiles = []
for x in range(int(min_x), int(max_x)+1):
    for y in range(int(min_y), int(max_y)+1):
        grid_tiles.append(str(x) + "_" + str(y))


distinct_floors = places[['building', 'floor']].drop_duplicates()
        
tessellations = pd.DataFrame(columns=['tile', 'building', 'floor', 'site'])
for i, row in distinct_floors.iterrows():
    tmp = pd.DataFrame(grid_tiles, columns=['tile'])
    tmp['building'] = row.building
    tmp['floor'] = row.floor
    tmp['site'] = 'NULL'
    tessellations = pd.concat([tessellations, tmp], axis=0)
    
    
tessellations['tessellation_type'] = 'grid'


tessellations['coord_a_x'] = tessellations.apply(lambda row: generate_tile_coords(row, 0), axis=1)
tessellations['coord_a_y'] = tessellations.apply(lambda row: generate_tile_coords(row, 1), axis=1)
tessellations['coord_b_x'] = tessellations.apply(lambda row: generate_tile_coords(row, 2), axis=1)
tessellations['coord_b_y'] = tessellations.apply(lambda row: generate_tile_coords(row, 3), axis=1)
tessellations['coord_c_x'] = tessellations.apply(lambda row: generate_tile_coords(row, 4), axis=1)
tessellations['coord_c_y'] = tessellations.apply(lambda row: generate_tile_coords(row, 5), axis=1)
tessellations['coord_d_x'] = tessellations.apply(lambda row: generate_tile_coords(row, 6), axis=1)
tessellations['coord_d_y'] = tessellations.apply(lambda row: generate_tile_coords(row, 7), axis=1)

    
tessellations = tessellations[['building','floor','site','tile','tessellation_type','coord_a_x','coord_a_y','coord_b_x','coord_b_y','coord_c_x','coord_c_y','coord_d_x','coord_d_y']]

for col in tessellations.columns:
    tessellations[col] = tessellations[col].astype(object)
    

tessellations.to_csv(work_path + '/tessellations.csv', sep=',', na_rep='NULL', index=False)

tessellations

,building,floor,site,tile,tessellation_type,coord_a_x,coord_a_y,coord_b_x,coord_b_y,coord_c_x,coord_c_y,coord_d_x,coord_d_y
0,1,0,NULL,117_49,grid,114.5,46.5,119.5,46.5,119.5,51.5,114.5,51.5
1,1,0,NULL,117_50,grid,114.5,47.5,119.5,47.5,119.5,52.5,114.5,52.5
2,1,0,NULL,117_51,grid,114.5,48.5,119.5,48.5,119.5,53.5,114.5,53.5
3,1,0,NULL,117_52,grid,114.5,49.5,119.5,49.5,119.5,54.5,114.5,54.5
4,1,0,NULL,117_53,grid,114.5,50.5,119.5,50.5,119.5,55.5,114.5,55.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25514,1,2,NULL,267_213,grid,264.5,210.5,269.5,210.5,269.5,215.5,264.5,215.5
25515,1,2,NULL,267_214,grid,264.5,211.5,269.5,211.5,269.5,216.5,264.5,216.5
25516,1,2,NULL,267_215,grid,264.5,212.5,269.5,212.5,269.5,217.5,264.5,217.5
25517,1,2,NULL,267_216,grid,264.5,213.5,269.5,213.5,269.5,218.5,264.5,218.5


In [54]:
# Generating the file adjacences.csv

# tile_1_building, tile_1_floor, tile_1_site, tile_1_tile, tile_2_building, tile_2_floor, tile_2_site, tile_2_tile, walkable, cost

all_tiles = tessellations[['building', 'floor', 'site','tile']].drop_duplicates().values

adjacences = []
for row in all_tiles:
    tile = row[3]
    x_cell = int(tile.split('_')[0])
    y_cell = int(tile.split('_')[1])
    
    neighbours = []
    for mod1 in [-1, 0, 1]:
        for mod2 in [-1, 0, 1]:
            neighbours.append((x_cell + mod1, y_cell + mod2))
  
    for neigh in neighbours: 
        if not(neigh[0] == x_cell and neigh[1] == y_cell) and neigh[0] >= int(min_x) and neigh[1] >= int(min_y) and neigh[0] <= int(max_x) and neigh[1] <= int(max_y):
            adjacences.append([str(row[0]), str(row[1]), 'NULL', tile, str(row[0]), str(row[1]), 'NULL', str(neigh[0]) + "_" + str(neigh[1]), 1, 'NULL'])
            

adjacences = pd.DataFrame(adjacences, columns=['tile_1_building', 'tile_1_floor', 'tile_1_site', 'tile_1_tile', 'tile_2_building', 'tile_2_floor', 'tile_2_site', 'tile_2_tile', 'walkable', 'cost'])

adjacences.to_csv(work_path + '/adjacences.csv', sep=',', na_rep='NULL', index=False)

adjacences

,tile_1_building,tile_1_floor,tile_1_site,tile_1_tile,tile_2_building,tile_2_floor,tile_2_site,tile_2_tile,walkable,cost
0,1,0,NULL,117_49,1,0,NULL,117_50,1,NULL
1,1,0,NULL,117_49,1,0,NULL,118_49,1,NULL
2,1,0,NULL,117_49,1,0,NULL,118_50,1,NULL
3,1,0,NULL,117_50,1,0,NULL,117_49,1,NULL
4,1,0,NULL,117_50,1,0,NULL,117_51,1,NULL
...,...,...,...,...,...,...,...,...,...,...
606703,1,2,NULL,267_216,1,2,NULL,267_215,1,NULL
606704,1,2,NULL,267_216,1,2,NULL,267_217,1,NULL
606705,1,2,NULL,267_217,1,2,NULL,266_216,1,NULL
606706,1,2,NULL,267_217,1,2,NULL,266_217,1,NULL


In [55]:
# Generating the file fingerprints.csv

fingerprints = dataframe[['fingerprint_id', 'coord_x', 'coord_y', 'coord_z', 'user_id', 'device_id', 'epoch', 'set', 'building', 'floor', 'site', 'tile']].copy()
fingerprints['tile_id'] = fingerprints.merge(tessellations, on=['building', 'floor', 'site', 'tile'], how='left', suffixes=('_1', ''))['tile'].fillna('NULL').values
fingerprints = fingerprints[['fingerprint_id', 'coord_x', 'coord_y', 'coord_z', 'building', 'floor', 'site', 'tile', 'user_id', 'device_id', 'epoch', 'set']]
fingerprints['user_id'] = [x if x != 0 else 'NULL' for x in fingerprints['user_id']]
fingerprints['is_radio_map'] = [True if x == 'training' else False for x in fingerprints['set']]
fingerprints['preceded_by'] = 'NULL'
fingerprints['followed_by'] = 'NULL'
fingerprints['notes'] = 'NULL'

for col in fingerprints.columns:
    fingerprints[col] = fingerprints[col].astype(object)
    

fingerprints.to_csv(work_path + '/fingerprints.csv', sep=',', na_rep='NULL', index=False)

fingerprints

,fingerprint_id,coord_x,coord_y,coord_z,building,floor,site,tile,user_id,device_id,epoch,set,is_radio_map,preceded_by,followed_by,notes
0,0,172.94,72.043,NULL,1,0,NULL,172_72,NULL,NULL,NULL,training,True,NULL,NULL,NULL
1,1,172.94,77.043,NULL,1,0,NULL,172_77,NULL,NULL,NULL,training,True,NULL,NULL,NULL
2,2,167.94,77.043,NULL,1,0,NULL,167_77,NULL,NULL,NULL,training,True,NULL,NULL,NULL
3,3,167.94,82.043,NULL,1,0,NULL,167_82,NULL,NULL,NULL,training,True,NULL,NULL,NULL
4,4,167.94,87.043,NULL,1,0,NULL,167_87,NULL,NULL,NULL,training,True,NULL,NULL,NULL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,1423,244.43,115.72,NULL,1,2,NULL,NULL,NULL,NULL,NULL,test,False,NULL,NULL,NULL
1424,1424,245.43,113.72,NULL,1,2,NULL,NULL,NULL,NULL,NULL,test,False,NULL,NULL,NULL
1425,1425,246.43,111.72,NULL,1,2,NULL,NULL,NULL,NULL,NULL,test,False,NULL,NULL,NULL
1426,1426,247.43,109.72,NULL,1,2,NULL,NULL,NULL,NULL,NULL,test,False,NULL,NULL,NULL


In [56]:
# Generating the file wifi_obs.csv

wifi_obs = dataframe[[x for x in dataframe.columns if 'WAP' in x]].copy()
wifi_obs['fingerprint_id'] = dataframe['fingerprint_id']
wifi_obs = wifi_obs[['fingerprint_id'] + [x for x in dataframe.columns if 'WAP' in x]]

for col in wifi_obs.columns:
    wifi_obs[col] = wifi_obs[col].astype(object)
    
wifi_obs.columns = ["AP-" + x + "-NULL" if "WAP" in x else x for x in list(wifi_obs.columns)]

wifi_obs.to_csv(work_path + '/wifi_obs.csv', sep=',', na_rep='NULL', index=False)

wifi_obs

,fingerprint_id,AP-WAP000-NULL,AP-WAP001-NULL,AP-WAP002-NULL,AP-WAP003-NULL,AP-WAP004-NULL,AP-WAP005-NULL,AP-WAP006-NULL,AP-WAP007-NULL,AP-WAP008-NULL,...,AP-WAP479-NULL,AP-WAP480-NULL,AP-WAP481-NULL,AP-WAP482-NULL,AP-WAP483-NULL,AP-WAP484-NULL,AP-WAP485-NULL,AP-WAP486-NULL,AP-WAP487-NULL,AP-WAP488-NULL
0,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1,1,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2,2,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
3,3,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
4,4,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1423,1423,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1424,1424,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1425,1425,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1426,1426,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


In [ ]:
#                                                                                           #                              
#                                                                                           #   
####################################### PART FOR TUT7 ####################################### 
#                                                                                           #   
#                                                                                           #   

In [57]:
dataset_name = 'TUT7' 


work_path = 'raw_db/' + dataset_name + '/'

file = open(work_path + '/dataset.pickle', 'rb')
dataframe = pickle.load(file)
file.close()


dataframe['coord_z'] = 'NULL'
dataframe['building_id'] = '1'
dataframe['site'] = 'NULL'
dataframe['tile'] = 'NULL'
dataframe['user_id'] = 'NULL'
dataframe['device_id'] = 'NULL'
dataframe['epoch'] = 'NULL'

dataframe[[x for x in dataframe.columns]] = dataframe[[x for x in dataframe.columns]].replace(100, 'NULL')

dataframe = dataframe[[x for x in dataframe.columns if 'WAP' in x] + ['coord_x', 'coord_y', 'coord_z', 'floor_id', 'building_id', 'site', 'tile', 'user_id', 'device_id', 'epoch', 'set']]
dataframe.reset_index(inplace=True)

dataframe.columns = ['fingerprint_id'] + [x for x in dataframe.columns if 'WAP' in x] + ['coord_x', 'coord_y', 'coord_z', 'floor', 'building', 'site', 'tile', 'user_id', 'device_id', 'epoch', 'set']

dataframe['tile'] = ['crowd_tile_1' if x == 'training' else 'NULL' for x in dataframe['set']]

        
dataframe

,fingerprint_id,WAP000,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,...,coord_y,coord_z,floor,building,site,tile,user_id,device_id,epoch,set
0,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,194.440,NULL,0,1,NULL,crowd_tile_1,NULL,NULL,NULL,training
1,1,NULL,-84,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,66.176,NULL,1,1,NULL,crowd_tile_1,NULL,NULL,NULL,training
2,2,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,144.700,NULL,2,1,NULL,crowd_tile_1,NULL,NULL,NULL,training
3,3,NULL,NULL,NULL,NULL,NULL,-65,NULL,NULL,NULL,...,200.860,NULL,1,1,NULL,crowd_tile_1,NULL,NULL,NULL,training
4,4,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,107.690,NULL,2,1,NULL,crowd_tile_1,NULL,NULL,NULL,training
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9286,9286,NULL,-76,NULL,NULL,NULL,NULL,NULL,NULL,-91,...,60.048,NULL,1,1,NULL,NULL,NULL,NULL,NULL,test
9287,9287,NULL,NULL,NULL,NULL,NULL,-69,NULL,NULL,NULL,...,199.690,NULL,1,1,NULL,NULL,NULL,NULL,NULL,test
9288,9288,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,176.100,NULL,0,1,NULL,NULL,NULL,NULL,NULL,test
9289,9289,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,195.890,NULL,1,1,NULL,NULL,NULL,NULL,NULL,test


In [58]:
# Generating the file places.csv

places = dataframe[['building', 'floor', 'site']].copy().drop_duplicates()
places.reset_index(inplace=True, drop=True)
places['floor_number'] = places['floor']
places = places[['building', 'floor', 'floor_number', 'site']]
places['site_height'] = 'NULL'
places['site_area'] = 'NULL'
places['floor_height'] = 3.5
places['floor_area'] = 'NULL'
places['building_area'] = 'NULL'

for col in places.columns:
    places[col] = places[col].astype(object)

places.to_csv(work_path + '/places.csv', sep=',', na_rep='NULL', index=False)

places

,building,floor,floor_number,site,site_height,site_area,floor_height,floor_area,building_area
0,1,0,0,NULL,NULL,NULL,3.5,NULL,NULL
1,1,1,1,NULL,NULL,NULL,3.5,NULL,NULL
2,1,2,2,NULL,NULL,NULL,3.5,NULL,NULL


In [59]:
# Generating the file tessellations.csv

tessellations = dataframe[['building', 'floor', 'site', 'tile']].copy().drop_duplicates()
tessellations['tessellation_type'] = 'crowd'
tessellations['coord_a_x'] = 'NULL'
tessellations['coord_a_y'] = 'NULL'
tessellations['coord_b_x'] = 'NULL'
tessellations['coord_b_y'] = 'NULL'
tessellations['coord_c_x'] = 'NULL'
tessellations['coord_c_y'] = 'NULL'
tessellations['coord_d_x'] = 'NULL'
tessellations['coord_d_y'] = 'NULL'
tessellations = tessellations[(tessellations['tile'] != 'NULL')]


for col in tessellations.columns:
    tessellations[col] = tessellations[col].astype(object)
    

tessellations.to_csv(work_path + '/tessellations.csv', sep=',', na_rep='NULL', index=False)

tessellations

,building,floor,site,tile,tessellation_type,coord_a_x,coord_a_y,coord_b_x,coord_b_y,coord_c_x,coord_c_y,coord_d_x,coord_d_y
0,1,0,NULL,crowd_tile_1,crowd,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1,1,1,NULL,crowd_tile_1,crowd,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2,1,2,NULL,crowd_tile_1,crowd,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


In [60]:
# Generating the file fingerprints.csv

fingerprints = dataframe[['fingerprint_id', 'coord_x', 'coord_y', 'coord_z', 'user_id', 'device_id', 'epoch', 'set', 'building', 'floor', 'site', 'tile']].copy()
fingerprints['tile_id'] = fingerprints.merge(tessellations, on=['building', 'floor', 'site', 'tile'], how='left', suffixes=('_1', ''))['tile'].fillna('NULL').values
fingerprints = fingerprints[['fingerprint_id', 'coord_x', 'coord_y', 'coord_z', 'building', 'floor', 'site', 'tile', 'user_id', 'device_id', 'epoch', 'set']]
fingerprints['user_id'] = [x if x != 0 else 'NULL' for x in fingerprints['user_id']]
fingerprints['is_radio_map'] = [True if x == 'training' else False for x in fingerprints['set']]
fingerprints['preceded_by'] = 'NULL'
fingerprints['followed_by'] = 'NULL'
fingerprints['notes'] = 'NULL'

for col in fingerprints.columns:
    fingerprints[col] = fingerprints[col].astype(object)
    

fingerprints.to_csv(work_path + '/fingerprints.csv', sep=',', na_rep='NULL', index=False)

fingerprints

,fingerprint_id,coord_x,coord_y,coord_z,building,floor,site,tile,user_id,device_id,epoch,set,is_radio_map,preceded_by,followed_by,notes
0,0,202.23,194.44,NULL,1,0,NULL,crowd_tile_1,NULL,NULL,NULL,training,True,NULL,NULL,NULL
1,1,208.02,66.176,NULL,1,1,NULL,crowd_tile_1,NULL,NULL,NULL,training,True,NULL,NULL,NULL
2,2,138.39,144.7,NULL,1,2,NULL,crowd_tile_1,NULL,NULL,NULL,training,True,NULL,NULL,NULL
3,3,176,200.86,NULL,1,1,NULL,crowd_tile_1,NULL,NULL,NULL,training,True,NULL,NULL,NULL
4,4,156.6,107.69,NULL,1,2,NULL,crowd_tile_1,NULL,NULL,NULL,training,True,NULL,NULL,NULL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9286,9286,194.94,60.048,NULL,1,1,NULL,NULL,NULL,NULL,NULL,test,False,NULL,NULL,NULL
9287,9287,173.64,199.69,NULL,1,1,NULL,NULL,NULL,NULL,NULL,test,False,NULL,NULL,NULL
9288,9288,143.46,176.1,NULL,1,0,NULL,NULL,NULL,NULL,NULL,test,False,NULL,NULL,NULL
9289,9289,205.78,195.89,NULL,1,1,NULL,NULL,NULL,NULL,NULL,test,False,NULL,NULL,NULL


In [61]:
# Generating the file wifi_obs.csv

wifi_obs = dataframe[[x for x in dataframe.columns if 'WAP' in x]].copy()
wifi_obs['fingerprint_id'] = dataframe['fingerprint_id']
wifi_obs = wifi_obs[['fingerprint_id'] + [x for x in dataframe.columns if 'WAP' in x]]

for col in wifi_obs.columns:
    wifi_obs[col] = wifi_obs[col].astype(object)
    
wifi_obs.columns = ["AP-" + x + "-NULL" if "WAP" in x else x for x in list(wifi_obs.columns)]

wifi_obs.to_csv(work_path + '/wifi_obs.csv', sep=',', na_rep='NULL', index=False)

wifi_obs

,fingerprint_id,AP-WAP000-NULL,AP-WAP001-NULL,AP-WAP002-NULL,AP-WAP003-NULL,AP-WAP004-NULL,AP-WAP005-NULL,AP-WAP006-NULL,AP-WAP007-NULL,AP-WAP008-NULL,...,AP-WAP791-NULL,AP-WAP792-NULL,AP-WAP793-NULL,AP-WAP794-NULL,AP-WAP795-NULL,AP-WAP796-NULL,AP-WAP797-NULL,AP-WAP798-NULL,AP-WAP799-NULL,AP-WAP800-NULL
0,0,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
1,1,NULL,-84,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
2,2,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
3,3,NULL,NULL,NULL,NULL,NULL,-65,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
4,4,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9286,9286,NULL,-76,NULL,NULL,NULL,NULL,NULL,NULL,-91,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
9287,9287,NULL,NULL,NULL,NULL,NULL,-69,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
9288,9288,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
9289,9289,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,...,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
